# Python for Data Analysis II

## Individual assignment

## Part 1 (regular expressions)

The goal is to extract dates of different formats from medical data.
We should correctly identify all of the different date variants encoded in this dataset and to properly standardize and sort the dates.

### Data loading

In [30]:
# Read the data
with open("./medical_dataset.txt") as f:
    lines = f.readlines()
    

### String vectorization

In [32]:
# Transform the data into a pandas dataframe

import pandas as pd
pd.options.display.max_rows = None

df = pd.DataFrame(lines, columns=["text"])

#Check number of rows
lend(df)

NameError: name 'lend' is not defined

### Steps

Each line of the file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

1- Have a look to the lines and take note of the different date formats in the file

2- Design and check a regular expression for each of these formats. Use vectorized strings in order to avoid loops

3- Try to rewrite these expressions more compactly (for example, by merging two or three regular expressions in one)

4- Create a dataframe with four columns: the original text, the month, the day and the year. All three fields must be numeric and the year must be represented by 4 digits. All texts must have this data extracted.

5- Save the final DataFrame to an excel file with name "processed_dates.xlsx"


### Tips

* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* There could be potential typos as this is a raw, real-life derived dataset.

**Example: detecting and extracting dates in mm/dd/yyyy format**

In [4]:
import re

# Create different regular expressions for types of date formats

expr_1 = r'(?:(\d{1,2}[-/]\d{1,2}[-/](?:\d{4}|\d{2})|' # 1. Date format e.g. mm/dd/yy, mm/dd/yyyy 
expr_2 = r'\d{1,2}[-/]\d{4}|' # 2. Date format e.g. mm/yyyy (without the date)
expr_3 = r'\d{1,2}[.,]? [A-Z][a-z]{2,}[,.]? (?:\d{4}|\d{2})|' # 3. Date format e.g. 12. September 2022 Month in letters
expr_4 = r'[A-Z][a-z]{2,}[,|.]?\s\d{1,2}[,.]?\s(?:\d{4}|\d{2})|' # 4. date format e.g. Sep. 12 2022 First month in letters and then date
expr_5 = r'[A-Z][a-z]{2,}[,.]? \d{4}' # 5. date format e.g. September 2022 Month in letters without date

# Merge all expressions into one regexp
final_expr = expr_1 + expr_2 + expr_3 + expr_4 + expr_5 + r'))'

# Extract the date as new column from the text by searching for the regexp
df["date"] = df["text"].str.extract(final_expr, expand = True)

# Show the new dataframe
df.head()

,text,date
0,03/25/93 Total time of visit (in minutes):\n,03/25/93
1,6/18/85 Primary Care Doctor:\n,6/18/85
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71
3,7 on 9/27/75 Audit C Score Current:\n,9/27/75
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/96


In [5]:
# Remove . or , from our date values
df['date'] = df.date.str.translate({ord(i): None for i in '.,'}) 
# Split the date column into month, day and year, remove all other signs like / and - 
df[['month','day','year']]  = df.date.str.split(r'[ /-]', expand = True) 

df

,text,date,month,day,year
0,03/25/93 Total time of visit (in minutes):\n,03/25/93,03,25,93
1,6/18/85 Primary Care Doctor:\n,6/18/85,6,18,85
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71,7,8,71
3,7 on 9/27/75 Audit C Score Current:\n,9/27/75,9,27,75
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/96,2,6,96
5,.Per 7/06/79 Movement D/O note:\n,7/06/79,7,06,79
6,"4, 5/18/78 Patient's thoughts about current su...",5/18/78,5,18,78
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,10/24/89,10,24,89
8,3/7/86 SOS-10 Total Score:\n,3/7/86,3,7,86
9,(4/10/71)Score-1Audit C Score Current:\n,4/10/71,4,10,71


In [20]:
# Assign the day, month and year into their correct respective columns

for i in range(len(df)):
    # check if day has 4 digit number, if so switch it with the value of year column
    if df.loc[i,'day'].isdigit() and len(df.loc[i,'day']) == 4:  
        df.loc[i, ['day', 'year']] = df.loc[i, ['year', 'day']].values 
    # check if day has letters and is not "None", if so swap with the value of the month column
    if df.loc[i,'day'] is not None and df.loc[i,'day'].isalpha():    
        df.loc[i, ['day', 'month']] = df.loc[i, ['month', 'day']].values 

In [21]:
# Uniform formatting of month, day and year

# Create a dictionary to decode month names into numbers
months_dict = { "jan" : 1,
       "feb" : 2,
       "mar" : 3,
       "apr" : 4,
       "may" : 5,
       "jun" : 6,
       "jul" : 7,
       "aug" : 8,
       "sep" : 9,
       "oct" : 10,
       "nov" : 11,
       "dec" : 12}  


for i in range(len(df)):
    # replace mising days with 1 as default
    if df.loc[i,'day'] is None: 
        df.loc[i,'day'] = 1
     # if year has only 2 digits, assume 19th century
    if len(df.loc[i,'year']) == 2:
        df.loc[i,'year'] = '19' + df.loc[i,'year']
    # convert month name to number using the dictionary months_dict
    if df.loc[i,'month'].isalpha():
        df.loc[i,'month'] = months_dict[df.loc[i,'month'][:3].lower()] 

#convert datatypes of month, day and year column to integer        
cols = ['month', 'day', 'year']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1) 

In [22]:
# Save to a new excel file
df.to_excel("processed_dates.xlsx")

## Part 2 (plotly)

In [6]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
py.init_notebook_mode(connected=True) # this allows to display plotly graphs in Jupyter

In [34]:
# Read dataset into a pandas dataframe and show its first rows
df = pd.read_csv("./dataset_housing.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 2.1. Is there any relation between neighborhood and price?


In [9]:
# Sort dataframe by increasing housing price
df = df.sort_values('SalePrice')
# Plot Neighborhoods against their distribution of housing price using box plot
fig = px.box(df,y="SalePrice",  x="Neighborhood", title="Distribution of housing prices of neighborhoods"
            , color_discrete_sequence=["orange"], orientation = 'v')

fig.show()

print("There is a relationship between Neighborhoods and Price, you can see that some neighborhoods are more expensive than others")

There is a relationship between Neighborhoods and Price, you can see that some neighborhoods are more expensive than others


## 2.2. Is there any relation between neighborhood and year built?



In [12]:
# Plot Neighborhoods against the distribution of the building years of their properties using strip plot
fig = px.strip(df, x="Neighborhood", y="YearBuilt",color_discrete_sequence=["green"] 
               ,title="Building year of properties across different neighborhoods")

py.iplot(fig)


print("There is a relationship between Neighborhoods and Year built, you can see that some neighborhoods have newer properties than others. Comparing this data with the relation with price, the neighborhoods with newer properties also are the more expensive ones.")

There is a relationship between Neighborhoods and Year built, you can see that some neighborhoods have newer properties than others. Comparing this data with the relation with price, the neighborhoods with newer properties also are the more expensive ones.


## 2.3. How overall quality, lot area, year built and price interact with each other?


In [16]:
# Create a scatter matrix, showing relationship of quality, lot area, year built and price

fig = go.Figure()
    
fig.add_trace(go.Splom(dimensions=[dict(label='Quality', values = df['OverallQual']),
                                   dict(label='Year Built', values = df['YearBuilt']),
                                   dict(label='Lot Area', values = df['LotArea']),
                                   dict(label='Price', values = df['SalePrice'])],
                       showupperhalf = False
                       ),
             )
fig.update_layout( title="Correlation of Quality, Lot Area, Year Built and Price", width = 1000, height = 1000)
fig.show()


print("Quality with Year Built: Quality shows an upward trend for newer properties")

print("Quality with Lot Area: Quality is almost evenly distributed for different lot areas")

print("Quality with Price: There is a clear trend visible, prices increase with higher quality")

print("Lot Area with Year Built: Lot area is almost evenly distributed for different years")

print("Lot Area with Price: Price is slightly increasing for larger lot areas, but as there are multiple outliers this trend is currently not easily visible.")

print("Year Built with Price: There is a clear trend visible, prices increase with newer properties")

Quality with Year Built: Quality shows an upward trend for newer properties
Quality with Lot Area: Quality is almost evenly distributed for different lot areas
Quality with Price: There is a clear trend visible, prices increase with higher quality
Lot Area with Year Built: Lot area is almost evenly distributed for different years
Lot Area with Price: Price is slightly increasing for larger lot areas, but as there are multiple outliers this trend is currently not easily visible.
Year Built with Price: There is a clear trend visible, prices increase with newer properties


## 2.4. How quality, lot area, year built and price interact with each other and evolve in time?

In [24]:
# Sort dataframe by increasing housing price
df = df.sort_values('YrSold')

fig1 = px.scatter(df, x="LotArea", y="YearBuilt", width=600, height=600, animation_frame="YrSold")
fig1.update_layout(xaxis_range=[0,50000])
fig1.show()

fig2 = px.scatter(df, x="LotArea", y="OverallQual", width=600, height=600, animation_frame="YrSold")
fig2.show()

fig3 = px.scatter(df, x="LotArea", y="SalePrice", width=600, height=600, animation_frame="YrSold")
fig3.update_layout(xaxis_range=[0,50000])
fig3.show()

fig4 = px.scatter(df, x="YearBuilt", y="OverallQual", width=600, height=600, animation_frame="YrSold")
fig4.show()

fig5 = px.scatter(df, x="YearBuilt", y="SalePrice", width=600, height=600, animation_frame="YrSold")
fig5.show()

fig6 = px.scatter(df, x="OverallQual", y="SalePrice", width=600, height=600, animation_frame="YrSold")
fig6.show()


print("The trend identified in 2.3. does not seem to change over the years where the properties were sold.")

The trend identified in 2.3. does not seem to change over the years where the properties were sold.
